In [1]:
import os
import sys
os.chdir('../')
SCRIPDIR = os.path.dirname(os.path.abspath("test.ipynb"))
sys.path.append(os.path.dirname(SCRIPDIR))
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def pick_a_sample(check_df: pd.DataFrame, result_df: pd.DataFrame = None, idx:int = None):
    if check_df[check_df['ERROR_REASON'].isna()].shape[0] == 0:
        print("No error found!")
        return None
    if idx is not None:
        sample = check_df.loc[[idx]]
    else:
        sample = check_df[check_df['ERROR_REASON'].isna() & check_df['IS_ERROR'] == True].sample()
    # if result_log is not None:
    print("-----------I have the schema generated as follows:---------")
        # logs = result_log.iloc[sample.LOGS_INDEX.values[0]]
    temp_df = result_df[result_df['TESTFILE_PATH'] == sample['TESTFILE_PATH'].values[0]]
    print(sample.index)
    for sql in temp_df[(temp_df['CASE_TYPE'] == 'Statement') | (temp_df.index == sample.index.values[0])]['SQL']:
        print(sql + ';')
        # print(logs.values[0])
    print("-----------The SQL commands is:---------")
    print(sample.SQL.values[0])
    print("-----------The expected result is:---------")
    print(sample.EXPECTED_RESULT.values[0])
    print("-----------The actual result is:---------")
    print(sample.ACTUAL_RESULT.values[0])
    print("-----------The error message is:---------")
    print(sample.ERROR_MSG.values[0])

    print("-----------The test file is:---------")
    print(sample.TESTFILE_PATH.values[0])
    print("___________The index is:___________")
    print(sample.index.values[0])
    return sample.index.values[0]

# SQLite

In [3]:
dbms_name = 'sqlite'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') & (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement')& (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

In [ ]:
pick_a_sample(check_df, df)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'TYPE_CAST': lambda x: re.match(r'unrecognized token: \":\"', str(x['ERROR_MSG'])) is not None,
    'NESTF': lambda x: re.search(r'range\((.*?)\)', str(x['SQL']), re.DOTALL | re.IGNORECASE) is not None,
    "GENERAL": lambda x: re.search(r'no such function: .*', str(x['ERROR_MSG']), re.IGNORECASE) is not None,
    "EXPLAIN": lambda x: re.match(r'EXPLAIN', str(x['SQL'])) is not None,
    # "SYNTAX": lambda x: re.search(r'syntax error', str(x['ERROR_MSG']), re.IGNORECASE) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
cnt-=1

In [ ]:
sample = sample_100.iloc[cnt]

print(sample['TESTFILE_PATH'])
print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [4]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [ ]:
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)

In [ ]:
des_cnt = visible_reasons.value_counts(['SUPER_TAG','DESCRIPTION'])
print(des_cnt)
des_cnt.to_csv('output/demo_sltddb_cnt.csv')

# PostgreSQL

In [5]:
dbms_name = 'postgresql'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [6]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') & (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement')& (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

0.4932963709677419
Queries: 6140, Statements: 3913
Wrong Queries: 667


In [ ]:
pick_a_sample(check_df, df)

In [8]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

Tag GENERAL is not in the test filter, implement it first!
Tag TYPE is not in the test filter, implement it first!
Tag SCHEMA is not in the test filter, implement it first!
Tag TYPENE is not in the test filter, implement it first!
Tag SQL is not in the test filter, implement it first!
Tag RANGE is not in the test filter, implement it first!
Tag SYNTAX is not in the test filter, implement it first!
Tag BOOL is not in the test filter, implement it first!
Tag NULLF is not in the test filter, implement it first!
Tag BIGFLOAT is not in the test filter, implement it first!
Tag OPERATOR is not in the test filter, implement it first!
Tag MISC is not in the test filter, implement it first!
Tag SYNTAX2 is not in the test filter, implement it first!
Tag EXPLAIN is not in the test filter, implement it first!
Result MisMatch:  667


In [9]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)
output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [10]:
sample_100.to_csv(output_path, index=False)

In [26]:
cnt=0

In [77]:
cnt-=1

In [136]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


-----------I have the schema generated as follows:---------
PRAGMA enable_verification;
CREATE VIEW struct_int AS SELECT * FROM (VALUES 	({'x': 1}, {'x': 1}), 	({'x': 1}, {'x': 2}), 	({'x': 2}, {'x': 1}), 	(NULL, {'x': 1}), 	({'x': 2}, NULL), 	(NULL, NULL) 	) tbl(l, r);
CREATE VIEW struct_str AS SELECT * FROM (VALUES 	({'x': 'duck'}, {'x': 'duck'}), 	({'x': 'duck'}, {'x': 'goose'}), 	({'x': 'goose'}, {'x': 'duck'}), 	(NULL, {'x': 'duck'}), 	({'x': 'goose'}, NULL), 	(NULL, NULL) 	) tbl(l, r);
CREATE VIEW struct_str_int AS SELECT * FROM (VALUES 	({'x': 'duck', 'y': 1}, {'x': 'duck', 'y': 1}), 	({'x': 'duck', 'y': 1}, {'x': 'goose', 'y': 2}), 	({'x': 'goose', 'y': 2}, {'x': 'duck', 'y': 1}), 	(NULL, {'x': 'duck', 'y': 1}), 	({'x': 'goose', 'y': 2}, NULL), 	(NULL, NULL) 	) tbl(l, r);
CREATE VIEW struct_nested AS SELECT * FROM (VALUES 	({'x': 1, 'y': {'a': 'duck', 'b': 1.5}}, {'x': 1, 'y': {'a': 'duck', 'b': 1.5}}), 	({'x': 1, 'y': {'a': 'duck', 'b': 1.5}}, {'x': 2, 'y': {'a': 'goose', 'b':

In [135]:
print(sample['TESTFILE_PATH'])

duckdb_tests/sql/generated_columns/virtual/group_by.test


In [139]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [140]:
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)

\begin{tabular}{lr}
 & DESCRIPTION \\
SUPER_TAG &  \\
Command & 16 \\
Function & 6 \\
IType & 3 \\
Misc & 10 \\
Setting & 3 \\
Type & 36 \\
UFunction & 26 \\
\end{tabular}



# MySQL

In [141]:
dbms_name = 'mysql'

output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [142]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_duckdb_filter_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_duckdb_filter_logs.csv")
error_percentage = df[df['IS_ERROR'] == True].shape[0] / df.shape[0]
print(1 - error_percentage)
error_queries = df[(df['CASE_TYPE'] == 'Query') &
                   (df['IS_ERROR'] == True)].shape[0]
error_statements = df[(df['CASE_TYPE'] == 'Statement') &
                      (df['IS_ERROR'] == True)].shape[0]
wrong_queries = df[(df['CASE_TYPE'] == 'Query') & (
    df['ERROR_MSG'] == 'Result MisMatch')].shape[0]
print(f"Queries: {error_queries}, Statements: {error_statements}")
print(f"Wrong Queries: {wrong_queries}")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()


0.3468916340263132
Queries: 7514, Statements: 5492
Wrong Queries: 366


In [143]:
pick_a_sample(check_df, df)

-----------I have the schema generated as follows:---------
Int64Index([10098], dtype='int64')
PRAGMA enable_verification;
CREATE TABLE test(a STRUCT(i INT, j INT));
SELECT a.* EXCLUDE(j) FROM test;;
CREATE TABLE a(i row(t int));
CREATE TABLE b(i row(t int));
SELECT i.* FROM a, b;
-----------The SQL commands is:---------
SELECT a.* EXCLUDE(j) FROM test;
-----------The expected result is:---------
1
-----------The actual result is:---------
False
-----------The error message is:---------
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'EXCLUDE(j) FROM test' at line 1
-----------The test file is:---------
duckdb_tests/sql/projection/select_struct_star.test
___________The index is:___________
10098


10098

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'TYPE_CAST': lambda x: re.match(r'1064 \(42000\).*\'::', str(x['ERROR_MSG'])) is not None,
    # 'GENERAL': lambda x: re.match(r'1305 \(42000\)', str(x['ERROR_MSG'])) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/duckdb_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['DESCRIPTION'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])
print("Result MisMatch: ", check_df[check_df['ERROR_MSG'] == 'Result MisMatch'].shape[0])

In [ ]:
errors = new_df[new_df['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)
output_path = f"output/{dbms_name}_duckdb_sample_100.csv"
output_reason_path = f"output/{dbms_name}_duckdb_sample_100_reason.csv"


In [ ]:
sample_100.to_csv(output_path, index=False)

In [ ]:
cnt=0

In [ ]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = new_df[new_df['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
print(sample['TESTFILE_PATH'])

In [ ]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[[
    'SQL', 'ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT', 'EXPECTED_RESULT',]]
reasons_mapping = pd.read_csv('data/duckdb_suite_errors.csv')
reasons_mapping = reasons_mapping[reasons_mapping['DBMS'] == dbms_name]
# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping,
                           right_on='TAG', left_on='ERROR_REASON', how='inner')
# visible_reasons.drop(columns=['TAG'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)


In [ ]:
visible_reasons = pd.read_csv(output_reason_path)
reason_summary = visible_reasons[['SUPER_TAG',  'DESCRIPTION']].groupby(['SUPER_TAG']).count().style.to_latex()
print(reason_summary)